In [2]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

# Properties

In [31]:
dataRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataLabeled'
featuresRoot = 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures'

fileName = 'return-01-loss-max-01-within-day.tsv'
fileNameDropedNA = 'return-01-loss-max-01-within-day-droped-na.tsv'
featuresFile = f'{featuresRoot}\\{fileName}'
featuresFileDropedNA = f'{featuresRoot}\\{fileNameDropedNA}'

In [3]:
file = f'{dataRoot}\\{fileName}'
data = pd.read_csv(file, sep='\t')
featured = data.copy()\
    .query("date > '2015-11-01'")\
    .sort_values('date')\
    .reset_index()\
    .drop('index', axis=1)

NameError: name 'dataRoot' is not defined

# Return

In [33]:
def lag(arr, l):
    laged = arr[:arr.shape[0]-l]
    laged = np.array([0]*l + laged.tolist())
    return laged

In [34]:
def _return(arr):
    arr0 = arr[:arr.shape[0]-1]
    arr1 = arr[1:]
    r = (arr1 / arr0) - 1
    r = np.insert(r, 0, 0., axis=0)
    return r

In [35]:
featured['return'] = _return(featured['close'].values)
featured['return_1'] = lag(featured['return'].values, 1)
featured['return_2'] = lag(featured['return'].values, 2)

# Z-Score

In [36]:
from scipy import stats
def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [37]:
featured['zscore_36_return'] = zscore(featured['return'], 36)
featured['zscore_288_return'] = zscore(featured['return'], 288)

In [38]:
featured['zscore_36_volume'] = zscore(featured['volume'], 36)
featured['zscore_288_volume'] = zscore(featured['volume'], 288)
featured['zscore_2016_volume'] = zscore(featured['volume'], 2016)

# Sliding Mean

In [39]:
def sliding_mean(x, window):
    return x.rolling(center=False,window=window).mean()

In [40]:
sliding_mean(featured['close'], 36)

0               nan
1               nan
2               nan
3               nan
4               nan
5               nan
6               nan
7               nan
8               nan
9               nan
10              nan
11              nan
12              nan
13              nan
14              nan
15              nan
16              nan
17              nan
18              nan
19              nan
20              nan
21              nan
22              nan
23              nan
24              nan
25              nan
26              nan
27              nan
28              nan
29              nan
            ...    
360262   6,496.8732
360263   6,496.6553
360264   6,495.7416
360265   6,494.8440
360266   6,494.4865
360267   6,493.9882
360268   6,493.5159
360269   6,492.6906
360270   6,492.5938
360271   6,492.5554
360272   6,492.4701
360273   6,493.0376
360274   6,493.4741
360275   6,493.6836
360276   6,493.5285
360277   6,493.5506
360278   6,493.5063
360279   6,492.5320
360280   6,491.4848


# Change In Price

$ pma = zscore(\frac{p}{avg(p, w_m)} - 1, w_z) $

In [41]:
def change_in_price(x, mean_window, zscore_window):
    mean_x = sliding_mean(x, mean_window)
    change_x = (x / mean_x) - 1
    zscore_change = zscore(change_x, zscore_window)
    return zscore_change

In [42]:
featured['pma36'] = change_in_price(featured['close'], 36, 288)
featured['pma288'] = change_in_price(featured['close'], 288, 288)
featured['pma2016'] = change_in_price(featured['close'], 2016, 288)

In [43]:
featured['vma36'] = change_in_price(featured['volume'], 36, 288)
featured['vma288'] = change_in_price(featured['volume'], 288, 288)
featured['vma2016'] = change_in_price(featured['volume'], 2016, 288)

# Sliding means change

$ ma = zscore(\frac{avg(p, w_1)}{avg(p, w_2)}-1, w_z) $

In [44]:
def sliding_means_change(x, w1, w2, zscore_window):
    mean1 = sliding_mean(x, w1)
    mean2 = sliding_mean(x, w2)
    change = (mean1 / mean2) - 1
    zscored = zscore(change, zscore_window)
    return zscored

In [45]:
featured['ma_12_108'] = sliding_means_change(featured['close'], 12, 108, 288)
featured['ma_36_288'] = sliding_means_change(featured['close'], 12, 288, 288)

# Price acceleration

$ pavg = \frac{p}{avg(p, w_m)} $

$ ac = zscore(\frac{pavg}{avg(pavg, w_m)}, w_z) $

In [46]:
def price_acceleration(x, w_mean, zscore_window):
    pavg = sliding_mean(x, w_mean)
    pavg_mean = sliding_mean(pavg, w_mean)
    acceleration = pavg / pavg_mean
    zscored = zscore(acceleration, zscore_window)
    return zscored

In [47]:
featured['ac36'] = price_acceleration(featured['close'], 36, 288)
featured['ac288'] = price_acceleration(featured['close'], 288, 288)

# Volatility

In [48]:
def std(x, window):
    return x.rolling(center=False,window=window).std()


def volatility(x, w_std, w_zscore):
    vol = std(x, w_std)
    return zscore(vol, w_zscore)

In [49]:
featured['vol36'] = volatility(featured['return'], 36, 288)
featured['vol288'] = volatility(featured['return'], 288, 288)
featured['vol2016'] = volatility(featured['return'], 2016, 288)

# Change in Volatility

$ sd = std(r, w_s)

$ dv =  zscore(\frac{std}{avg(std, w_a)}, w_z) $

In [50]:
def volatility_change(x, w_std, w_a,  w_zscore):
    sd = std(x, w_std)
    sd_mean = sliding_mean(sd, w_a)
    change = sd / sd_mean
    return zscore(change, w_zscore)

In [51]:
featured['dv_36_288'] = volatility_change(featured['return'], 36, 288, 288)
featured['dv_288_2016'] = volatility_change(featured['return'], 288, 2016, 288)

In [52]:
featured.head()

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,...,vma2016,ma_12_108,ma_36_288,ac36,ac288,vol36,vol288,vol2016,dv_36_288,dv_288_2016
0,1424372400,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 20:00:00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1424373000,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 20:10:00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1424373300,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:15:00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,1424373600,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:20:00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1424373900,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 20:25:00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# Save Features

In [53]:
featured.columns

Index(['timestamp', 'period', 'open', 'high', 'low', 'close', 'volume',
       'quoteVolume', 'weightedAverage', 'date', 'maxReturn', 'tillMax',
       'minReturn', 'label', 'return', 'return_1', 'return_2',
       'zscore_36_return', 'zscore_288_return', 'zscore_36_volume',
       'zscore_288_volume', 'zscore_2016_volume', 'pma36', 'pma288', 'pma2016',
       'vma36', 'vma288', 'vma2016', 'ma_12_108', 'ma_36_288', 'ac36', 'ac288',
       'vol36', 'vol288', 'vol2016', 'dv_36_288', 'dv_288_2016'],
      dtype='object')

In [54]:
write_tsv(featured.fillna(0), featuresFile)
write_tsv(featured.dropna(), featuresFileDropedNA)

In [55]:
featured.shape

(360292, 37)

In [56]:
featured.dropna().shape

(324903, 37)